# Example on how to create a component

Let's look at how to create a component for a predefined pipeline. We are going to be working with the stock_price_prediction pipeline which can be found in the notebook of the same name in this folder. You can run through the code and read the comments to get a general sense of what the pipeline does. The script reads data from a csv file, cleans the data and groups it. Next the script trains a linear regression model based on the data. The script tests the model by doing some predicitons, and lastly it draws some graphs displaying the result. 

We are going to split this script into three microservices, one for data handling, one for training and one for testing. The script could be split into both more or less microservices, depending on your needs. 

### Defining the service
Let's start with defining the data handling microservice. Based on the stock price prediction script, we know that the data handling component needs to:

1) read data from a csv file
2) remove NaN values
3) define target and feature variables
4) split the data into training and testing sets

Let's first define necessary functions for completing these steps. We want to define the steps as functions instead of a script in order to use gRPC. These steps can be split into separate functions, but for this example we are going to combine all of them into one function to simplify the example and keep the amount of code minimal. 

Here is one example of a function that completes the necessary steps: 

```python
import pandas as pd
from sklearn.model_selection import train_test_split

def clean_data(csv_file):
    data = pd.read_csv(csv_file)
    data['Date'] = pd.to_datetime(data['Date'])
    data['Previous_Close'] = data['Close'].shift(1)
    data = data.dropna()
    
    x = data[['Previous_Close']]
    y = data['Close']
    dates = data['Date']
    
    x_train, x_test, y_train, y_test, dates_train, dates_test = train_test_split(
        x, y, dates, test_size=0.2, shuffle=False
    )
    
    # Flatten the lists
    x_train_flat = [item for sublist in x_train.values for item in sublist]
    x_test_flat = [item for sublist in x_test.values for item in sublist]
    y_train_flat = y_train.tolist()
    y_test_flat = y_test.tolist()
    dates_train_str = dates_train.dt.strftime('%Y-%m-%d').tolist()
    dates_test_str = dates_test.dt.strftime('%Y-%m-%d').tolist()
    
    return x_train_flat, x_test_flat, y_train_flat, y_test_flat, dates_train_str, dates_test_str

You might have noticed that we have flattened the x_train and x_test lists. This has mainly been done to simplify the transfer of data with grpc, as creating lists of lists can be a bit of a hassle with protobuf. 

This code can now either be saved into it's own file, like for example data_service.py, or it can be later added into the server file, for example data_server.py. In the example provided in the folder "1.4connecting_services_with_grpc, we have first defined a service and then later imported the function from the service file into the server file. 

### Creating the protofile

Next step is to define the protofile for the service. To define the protofile we need to consider what data will be passed between the server and the client. We want the client to send a csv file to the server, and this can be done in a few different ways, either as a string or as binary data. For this example we have chosen binary data, so we know that we need one message type for this. The function returns six different lists, so we also need to define a message type that can hold the lists. Here is an example of how to define these messages:

```proto
syntax = "proto3";

package data;

// Request message containing CSV data as bytes.
message DataRequest {
    bytes csv_content = 1;
}


// Response message containing cleaned data.
message DataResponse {
    repeated double x_train = 1;
    repeated double x_test = 2;
    repeated double y_train = 3;
    repeated double y_test = 4;
    repeated string dates_train = 5;
    repeated string dates_test = 6;
}


// The DataService definition.
service DataService {
    // Sends CSV content and receives cleaned data.
    rpc CleanData (DataRequest) returns (DataResponse);
}
```
Here you can see that the keyword "repeated" has been used when the datatype is a list. 

### Generating the grpc code

Now we have all the necessary components for the protofile. Next we need to generate the grpc code from the protofile. This can be done in the folder containing the protofile with the following command:

```bash
python3 -m grpc_tools.protoc -I./ --python_out=. --grpc_python_out=. data.proto
```
This should generate two files with code that you can use for defining the server and the client. The files should have the suffixes _pb2.py and _pb2_grpc.py. The first file contains message classes for the messages you defined in the protofile. There are two messageclasses, as we have two messages, and those can be accessed with data_pb2.DataRequest and data_pb2.DataResponse. The second file contains the necessary code for creating the client and the server for the microservice, which is the next step. 

### Creating the server
Let's first create the server for the microservice. For this you are going to want to utilize some of the code from the generated grpc files. In the file ending with _pb2_grpc.py you should find class definitions for both a servicer and a service, which you are going to need when creating the server. 

The Servicer is a class that you implement to handle the server-side logic of your gRPC service. This is where you define how each RPC method should behave by writing the actual business logic for the methods described in your .proto file. You need to create a subclass of the generated DataServiceServicer and implement the RPC methods defined in the .proto file. The Service class is used internally by gRPC to provide methods to interact with the RPCs. It's auto-generated by protoc and typically includes static methods for the client to call RPCs and functions to add handlers to the server. The Service is not typically modified directly.

To implement the server you should follow these steps:
1) Implement the Servicer: Create a subclass of the generated DataServiceServicer and implement the methods.
2) Start the Server: Use the add_DataServiceServicer_to_server function to attach your servicer to the server and start it.

Since we already wrote the function clean_data in the service file, all you have to do is import it and possibly add some error handling to implement the RPC function CleanData. You also need to make sure that the return type of the RPC method follows the definition in the protofile. We defined the return type to be a collection of three values, previous_close, close and dates, so you need to make sure that the returned value follows this definiton. Here is an example of how to create the subclass for the servicer:

```python
# data_service_server.py
from concurrent import futures
import grpc
from data import data_pb2_grpc
from data import data_pb2
import pandas as pd
from data.data_service import clean_data
import os
import io
import logging

class DataServiceServicer(data_pb2_grpc.DataServiceServicer):
    def CleanData(self, request, context):
        try:
            # Read CSV content from the request
            csv_content = request.csv_content
            csv_file = io.BytesIO(csv_content)
            
            logging.info("Received CSV data, cleaning...")
            # Clean data
            x_train, x_test, y_train, y_test, dates_train, dates_test = clean_data(csv_file)
            logging.info("Data cleaned successfully")

            print(x_train, x_test, y_train, y_test, dates_train, dates_test)

            
            return data_pb2.DataResponse(
                x_train=x_train,
                x_test=x_test,
                y_train=y_train,
                y_test=y_test,
                dates_train=dates_train,
                dates_test=dates_test
            )
        except Exception as e:
            logging.exception("Error cleaning data")
            context.set_code(grpc.StatusCode.INTERNAL)
            context.set_details(f"Internal error: {str(e)}")
            return data_pb2.DataResponse()


```

Here you can see that we first read the csv content and save it to a csv file. The file can then be passed to the clean_data function that we wrote earlier. Lastly we make sure to save the values in the correct return format, data_pb2.DataResponse. We have also added some error handling for better performance.

Now we need to complete the second step for creating the server, start the server. For this we can use the generated function add_DataServiceServer. In order to do this we need to first create a grpc server, that can be done using the following line:

```python
server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
```
futures.ThreadPoolExecutor(max_workers=10) specifies the thread pool executor for handling concurrent RPCs. ThreadPoolExecutor manages a pool of threads for executing tasks asynchronously. Here, it allows the server to handle up to 10 concurrent requests (RPC calls) in parallel.

Now that we have a server, we can add the servicer to it using the following:

```python
data_pb2_grpc.add_DataServiceServicer_to_server(DataServiceServicer(), server)
```

Lastly we need to define a port, start the server and make sure it stays running. This can be done like this:

```python
server.add_insecure_port('[::]:50051')
server.start()
server.wait_for_termination()
``` 
Now you have all the necessary components for your server. You can take a look at the data_service_server file to see the final code for the server. This file can be found in the "1.4connecting_services_with_grpc" folder.

### Creating the client

Lastly we need to create a client in order to interact with our server. Creating a gRPC client involves setting up a communication channel, creating a stub to interact with the server, making a request, and handling the response. You are going to want to use the stub that was generated from the protofile. The stub acts as an intermediary between your application code and the remote gRPC service. The stub provides methods that correspond to the RPCs defined in the .proto file, allowing your application to invoke these methods as if they were local functions, even though they are executed on a remote server. 

First we are going to create a channel, a communication path, to the server. You need to specify the server's address (in this case, localhost:50051) in order to create the channel. This can be done like so:

```python
with grpc.insecure_channel('localhost:50051') as channel:
```
The with statement ensures the channel is properly closed when the operation is done.

Next we will define the stub using the channel:

```python
stub = data_pb2_grpc.DataServiceStub(channel)
```

Now let's read and prepare the data for the request. Reading the file as bytes prepares the data to be sent in the DataRequest as defined in the protofile. This is how the reading can be done:

```python
csv_file_path = './MSFT.US.csv'

with open(csv_file_path, 'rb') as f:
    csv_content = f.read()
```

Next we need to construct a request message using the data read in the previous step. The request message format is defined in the .proto file. 

```python
request = data_pb2.DataRequest(csv_content=csv_content)
```
Now we can invoke the remote method on the stub. This sends the request to the server and waits for a response.

```python
response = stub.CleanData(request)
```
Now we can check the server’s response and process it as needed. This could involve printing the response data or using it in further computations.

```python
if response.x_train and response.x_test and response.y_train and response.y_test and response.dates_train and response.dates_test:
    print("x_trian:", response.x_train)
    print("x_test:", response.x_test)
    print("y_train:", response.y_train)
    print("y_test:", response.y_test)
    print("Dates Train:", response.dates_train)
    print("Dates Test:", response.dates_test)
else:
    print("No data returned or some fields are empty.")
```

These are all the necessary parts of the client. You can see the entire client file (data_client.py) in the "1.4connecting_services_with_grpc" folder as well. 


### Testing
Now that you have all the necessary files, you can test the server and client. You can run the server with the command python3 data_service_server.py and the same goes for the client. You should see the cleaned data being printed on the client side. 

Now you could continue by creating the other services in the same way:
    1) define the service
    2) write protofile
    3) generate grpc code
    4) create server and client

We have already done this for the training and testing, and included all the clients into one file called client.py. If you run all the servers and then run the client you should be able to see how the whole pipeline is executed and a graph is produced as output. 